In [1]:
from binary_param import linear, constant
import pandas as pd
import numpy as np
from HFCs_bmimBF4_PR import configuration
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest
from pyomo.opt import TerminationCondition, SolverStatus

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Flash
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
m1 = ConcreteModel()

m1.fs = FlowsheetBlock(default={"dynamic": False})

m1.fs.properties = GenericParameterBlock(default=configuration)

m1.fs.F101 = Flash(default={"property_package": m1.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

In [3]:
data = pd.read_csv('R32_bmimbf4_09_2.csv')
data2 = pd.read_csv('R125_bmimbf4_09_2.csv')

In [4]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [5]:
# Make outlet arrays
IL_x = np.zeros(len(data["1"]))
IL_y = np.zeros(len(data["1"]))
R32_x = np.zeros(len(data["1"]))
R32_y = np.zeros(len(data["1"]))
R125_x = np.zeros(len(data["1"]))
R125_y = np.zeros(len(data["1"]))

alpha1 = np.zeros(len(data["1"]))

m1.fs.F101.inlet.flow_mol.fix(10)
m1.fs.F101.inlet.temperature.fix(298)
m1.fs.F101.inlet.mole_frac_comp[0, "bmimBF4"].fix(0.5) 
m1.fs.F101.inlet.pressure.fix(100000)
m1.fs.F101.vap_outlet.pressure.fix(100000)
m1.fs.F101.vap_outlet.temperature.fix(298.15)

for i in range(len(data["0"])):
    
    m1.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.009)
    m1.fs.properties.PR_kappa_A['R125', 'R32'].fix(0.007)
    m1.fs.properties.PR_kappa_A['R32', 'bmimBF4'].fix(data["1"].iloc[i])
    m1.fs.properties.PR_kappa_A['bmimBF4', 'R32'].fix(data["0"].iloc[i])
    m1.fs.properties.PR_kappa_A['bmimBF4', 'R125'].fix(data2["0"].iloc[i])
    m1.fs.properties.PR_kappa_A['R125', 'bmimBF4'].fix(data2["1"].iloc[i])

    m1.fs.F101.inlet.flow_mol.fix(10)
    m1.fs.F101.inlet.temperature.fix(298)
    m1.fs.F101.inlet.mole_frac_comp[0, "bmimBF4"].fix(0.7) 
    m1.fs.F101.inlet.mole_frac_comp[0, "R32"].fix(0.2) 
    m1.fs.F101.inlet.mole_frac_comp[0, "R125"].fix(0.1)

    m1.fs.F101.inlet.pressure.fix(100000)

    m1.fs.F101.vap_outlet.pressure.fix(100000)
    m1.fs.F101.vap_outlet.temperature.fix(298.15)

    try:
        m1.fs.F101.initialize(outlvl=idaeslog.CRITICAL)              

        # solve the model
        status = solver.solve(m1, tee = False)

        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):

            IL_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "bmimBF4"])
            IL_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "bmimBF4"])
            R32_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "R32"]) 
            R32_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "R32"])
            R125_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "R125"])
            R125_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "R125"])

            print(i)
            
            alpha1[i] = (R125_y[i]/R125_x[i])/(R32_y[i]/R32_x[i])

    except ValueError:
        alpha1[i] = 'NaN'
       
        
results = pd.DataFrame(alpha1)
results.to_csv('bmimbf4_alpha_10_2.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [6]:
print(alpha1)

[2.64925114 2.57724996 2.61559011 2.74389857 3.17996564 4.34475779
 4.56225067 2.69990911 2.82872441 3.10825304 2.51514659 2.70367199
 2.71392252 2.92930072 3.61926306 3.1011516  2.68798322 2.84007534
 2.39188668 2.76066898 2.89004201 3.09228272 2.75806353 2.81641236
 2.81939213 4.48625326 2.98918956 3.22052858 2.98238714 2.58599366
 2.73072697 3.51369393 4.4432371  2.77764076 4.43627581 3.03002284
 3.2102771  4.4507964  3.23408319 3.19359722 2.76348404 4.53421275
 2.81950135 2.93900671 2.7004572  3.94201959 2.55633586 3.1443332
 2.53504477 2.88632302 2.47031358 4.40592171 3.23109447 2.18233768
 2.57197376 2.69062739 2.85494118 2.88047992 2.68249614 4.10432896
 2.42644471 3.52344923 3.12448675 2.99736182 2.67250694 2.49993209
 2.59107229 2.49092025 5.0969712  2.53828673 2.96577416 2.3746618
 3.08475678 2.70862095 3.98431404 2.55241843 2.87440236 2.98825649
 2.95267571 2.84493161 2.62586929 4.36901688 2.97554561 2.91123317
 2.71025    3.33929053 3.27441773 2.84700526 4.86949247 2.136116

In [7]:
# results = pd.DataFrame(alpha1)
# results.to_csv('alpha1_07_bf4.csv')